# Smart Device API Code Generation Hackathon

In this hackathon, we will be generating with AI/ML provided by the well established [computate project](https://github.com/computate-org/computate). We will automatically generate OpenAPI specs, database table schemas, Java POJOs, Vert.x Reactive Java APIs, Handlebars HTML page templates, JavaScript page and API functions, NGSI-LD Context data, OpenShift Custom Resource Definitions, and Ansible Operator roles and playbooks for any of the hundreds of open source Edge device data related [FIWARE Smart Data Models available here](https://github.com/smart-data-models). 

## Prerequisites
To run the AI/ML code generation, you need to have completed the steps in the [README](README.md), [notebook 01-install-prerequisites.ipynb](01-install-prerequisites.ipynb), and [ notebook 02-deploy-microservices.ipynb](01-install-prerequisites.ipynb) to install the dependencies into the workbench, and deploy all the microservices. Then run the commands below in your OpenShift AI Workbench. 

## Clone Computate and Smart Village repos

### computate_project Ansible Role

The [computate_project Ansible Role](https://github.com/computate-org/computate_project.git) is for installing Java projects that want to generate code for them using the [computate project](https://github.com/computate-org/computate.git). 

In [ ]:
install -d ~/.ansible/roles
git clone git@github.com:computate-org/computate_project.git ~/.ansible/roles/computate.computate_project
echo DONE

### computate project

The [computate project](https://github.com/computate-org/computate.git) is a Java project that watches for changes to files in a directory recursively, parses the Java code as it's created or updated, indexes every detail about each Java class, constructor, method, and field in the Apache Solr search engine, and generates code based on what it discovers. This is the main open source library that does the AI/ML code generation. Instead of a typical model server, we use Apache Solr as the model server. We can flexibly index every aspect about our Java code in Solr as we code over time, including multiple Java projects at the same time, and link together classes, types, and foreign key relations between projects. 

In [ ]:
git clone https://github.com/computate-org/computate.git ~/computate
echo DONE

Use maven to compile and install the computate project in the workbench. 

In [ ]:
(cd ~/computate &&  mvn clean install)
echo DONE

### computate-search project

The [computate-search project](https://github.com/computate-org/computate-search.git) is a Java project that mainly interacts with Apache Solr to make search queries, and parse the response. There are also several date time serializers and deserializers for handling date times in requests. One other important class is the `Wrap` class which can wrap a Java field initialization value by it's generic type as part of the generated initialization code for each class. 

In [ ]:
git clone https://github.com/computate-org/computate-search.git ~/computate-search
echo DONE

Run the Ansible Playbook to configure and compile the `computate-search` project. 

In [ ]:
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=computate-search \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/computate-search" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

Run the `index.sh` shell script of the `computate` project on  the  `computate-search` project to index all of the `computate-search` Java classes into the Apache Solr search engine. 

In [ ]:
env SITE_NAME=computate-search \
  SITE_PATH=$HOME/computate-search \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

Now query the Solr search engine to find out how many Java classes, constructors, methods, fields, and generated fields are found in the  `computate-search` project. 

In [ ]:
curl -s 'http://solr:8983/solr/computate/query?rows=0&fq=siteNom_indexed_string:computate-search' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

In [ ]:
git clone https://github.com/computate-org/computate-vertx.git ~/computate-vertx
echo DONE

In [30]:
curl -s 'http://solr:8983/solr/computate/query?rows=0&fq=siteNom_indexed_string:computate-vertx' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

34 Java classes (for example ComputateJavaClass, ComputateConfigKeys, ApiWriter, ComputatePageLayout, SearchList)
7 constructors
254 methods (for example create, serialize, toString, deserialize, toId)
238 fields (for example LOG, formatZonedDateTime, templateEngine, ACCOUNT_EMAIL, API_BASE_PATH)
360 new generated fields (for example siteRequest_, classSimpleName, config, languageName, appName)
DONE


In [ ]:
git clone https://github.com/computate-org/smartvillage-platform.git ~/smartvillage-platform
echo DONE

In [29]:
curl -s 'http://solr:8983/solr/computate/query?rows=0&fq=siteNom_indexed_string:smartvillage-platform' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

24 Java classes (for example PageLayout, ConfigKeys, SiteRequestEnUS, AsyncInputStream, SystemEvent)
4 constructors
61 methods (for example toId, doRead, _id, _objectId, _objectTitle)
73 fields (for example API_COUNTER_FETCH_SiteHtm, API_COUNTER_FETCH_SitePage, API_COUNTER_FETCH_TimeStep, API_COUNTER_FETCH_VehicleStep, API_COUNTER_RESUME_TimeStep)
253 new generated fields (for example siteRequest_, userKey, classSimpleName, pageId, sessionId)
DONE


In [ ]:
git clone https://github.com/computate-org/smartvillage-platform-static.git ~/smartvillage-platform-static
echo DONE

In [ ]:
git clone https://github.com/computate-org/smartabyar-smartvillage.git ~/smartabyar-smartvillage
echo DONE

In [31]:
curl -s 'http://solr:8983/solr/computate/query?rows=0&fq=siteNom_indexed_string:smartabyar-smartvillage' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

114 Java classes (for example TrafficSimulationGen, StorageBatteryDeviceGen, TrafficFlowObservedGen, SmartTrafficLightGen, StorageBatteryMeasurementGen)
17 constructors
6496 methods (for example displayNameForClass, obtainForClass, relateForClass, staticSearchForClass, staticSearchFqForClass)
1771 fields (for example LOG, CLASS_SIMPLE_NAME, DISPLAY_NAME_id, VAR_id, id)
663 new generated fields (for example id, location, pk, areaServed, areaServedColors)
DONE


In [ ]:
git clone https://github.com/computate-org/smartabyar-smartvillage-static.git ~/smartabyar-smartvillage-static
echo DONE

In [ ]:
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=computate-vertx \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/computate-vertx" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

In [ ]:
env SITE_NAME=computate-vertx \
  SITE_PATH=$HOME/computate-vertx \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

In [ ]:
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=smartvillage-platform \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/smartvillage-platform" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

In [ ]:
env SITE_NAME=smartvillage-platform \
  SITE_PATH=$HOME/smartvillage-platform \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

In [ ]:
SITE_HOST_NAME="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get route/sumodev -o jsonpath={.spec.host})"
AUTH_SECRET_NAME="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/keycloak-client-secret-smartvillage -o jsonpath={.data.CLIENT_SECRET})"
AUTH_CLIENT="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/keycloak-client-secret-smartvillage -o jsonpath={.data.CLIENT_ID} | base64 -d)"
AUTH_SECRET="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/keycloak-client-secret-smartvillage -o jsonpath={.data.CLIENT_SECRET} | base64 -d)"

oc extract -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) secret/smartvillage-kafka --to=$HOME/smartabyar-smartvillage/config/ --keys=user.p12 --confirm
KAFKA_SSL_KEYSTORE_PASSWORD="$(oc get -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) secret/smartvillage-kafka -o 'jsonpath={.data.user\.password}' | base64 -d)"
oc extract -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) secret/default-cluster-ca-cert --to=$HOME/smartabyar-smartvillage/config/ --keys=ca.p12 --confirm
KAFKA_SSL_TRUSTSTORE_PASSWORD="$(oc get -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) secret/default-cluster-ca-cert -o 'jsonpath={.data.ca\.password}' | base64 -d)"

ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=smartabyar-smartvillage \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME_COMPUTATE=solr \
  -e SOLR_PORT_COMPUTATE=8983 \
  -e SOLR_SSL_COMPUTATE=false \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/smartabyar-smartvillage" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e ZOOKEEPER_HOST_NAME=zookeeper \
  -e ZOOKEEPER_PORT=2181 \
  -e JDBC_HOST=postgres-primary \
  -e JDBC_PORT=5432 \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SITE_BASE_URL="https://${SITE_HOST_NAME}" \
  -e STATIC_BASE_URL="https://${SITE_HOST_NAME}/static" \
  -e AUTH_CLIENT="${AUTH_CLIENT}" \
  -e AUTH_SECRET="${AUTH_SECRET}" \
  -e KAFKA_SSL_KEYSTORE_PASSWORD="${KAFKA_SSL_KEYSTORE_PASSWORD}" \
  -e KAFKA_SSL_TRUSTSTORE_PASSWORD="${KAFKA_SSL_TRUSTSTORE_PASSWORD}" \
  -e KAFKA_BROKERS="default-kafka-bootstrap:9093" \
  -e ENABLE_IMPORT_DATA=false
echo DONE

In [ ]:
oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) rsync \
  /opt/app-root/src/smartabyar-smartvillage/src/main/resources/sql/ \
  $(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get pod -l postgres-operator.crunchydata.com/role=master -o name):/tmp/
oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) exec \
  $(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get pod -l postgres-operator.crunchydata.com/role=master -o name) \
  -- bash -c 'psql -U smartvillage smartvillage < /tmp/db-create.sql'

In [ ]:
env SITE_NAME=smartabyar-smartvillage \
  SITE_PATH=$HOME/smartabyar-smartvillage \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/generate.sh
echo DONE

In [ ]:
echo '
kind: Service
apiVersion: v1
metadata:
  name: smartvillage-devel
  namespace: $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
spec:
  ports:
    - protocol: TCP
      port: 12080
      targetPort: 12080
  type: ClusterIP
  selector:
    notebook-name: sumo
' | oc apply -f -
